In [ ]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib_venn import venn2

from src.features import build_features

sns.set_theme(palette="pastel")
plt.rcParams["figure.figsize"] = (12, 6)
pd.set_option("display.max_columns", None)

COLUMNAS_GRD = [
    "RUT_LIMPIO",
    "SEXO",
    # "EDAD_CANT",
    "PREVI",
    "BENEF",
    "NOM_HOPI_P",
    "FECHA_INGRESO",
    "FECHA_EGRESO",
    "ANO_EGR",
    "DIAS_ESTAD",
    "DIAG1",
    "DIAG2",
    "DIAG3",
    "INTERV_Q",
    "COD_FONASA",
    "INTERV_Q_PPAL",
    "INTERV_Q_2",
    "INTERV_Q_3",
    "T_DIAG1",
    "T_DIAG2",
    "T_DIAG3",
    "PROCED",
    "PROCED_PPAL",
    "PROCED_2",
    "PROCED_3",
]

COLUMNA_GRD_INTERNO = [
    "rut",
    "RUT_LIMPIO",
    "sexo_cod",
    "prevision_cod",
    "hospital_de_procedencia_cod",
    "fecha_ingreso",
    "fecha_de_egreso",
    "ano_de_egreso",
    "estancias_totales",
    "diag_01_principal_cod",
    "diag_01_principal_des",
    "diag_02_secundario_cod",
    "diag_02_secundario_des",
    "diag_03_secundario_cod",
    "diag_03_secundario_des",
    "proced_01_principal_cod",
    "proced_01_principal_des",
    "1ra_intervencion_cirujano",
    "especialidad_medica_de_la_intervencion_des",
]

In [ ]:
RUTA_SABANAS = "../data/processed/grd_sabanas.csv"
COLUMNAS_FECHA_SABANAS = ["FECHA_INGRESO", "FECHA_EGRESO"]

RUTA_GRD_INTERNO = "../data/processed/grd_interno.csv"
COLUMNAS_FECHA_GRD_INT = ["fecha_ingreso", "fecha_de_egreso"]

RUTA_PABELLON = "../data/external/datos_limpios_pabellon.csv"

# Leer y formatear datos de GRD
df_grd_sabanas = build_features.cargar_datos_grd(RUTA_SABANAS, columnas=COLUMNAS_GRD)
df_grd_sabanas = build_features.formatear_columnas_fecha(df_grd_sabanas, COLUMNAS_FECHA_SABANAS)
df_grd_sabanas["RUT_LIMPIO"] = df_grd_sabanas["RUT_LIMPIO"].astype(str)

df_grd_interno = build_features.cargar_datos_grd(RUTA_GRD_INTERNO, columnas=COLUMNA_GRD_INTERNO)
df_grd_interno = build_features.formatear_columnas_fecha(df_grd_interno, COLUMNAS_FECHA_GRD_INT)
df_grd_interno = df_grd_interno.reset_index(names="id_egreso")
df_grd_interno["RUT_LIMPIO"] = df_grd_interno["RUT_LIMPIO"].astype(str)
# Cambia nomenclatra del codigo CIE-10
df_grd_interno["diag_01_principal_cod"] = (
    df_grd_interno["diag_01_principal_cod"].str.replace(".", "", regex=False).str.ljust(4, "X")
)

# Leer y formatear datos de operaciones quirúrgicas
df_pabellon = pd.read_csv(RUTA_PABELLON, encoding="latin-1", sep=";")
df_pabellon["fecha"] = pd.to_datetime(df_pabellon["fecha"])
df_pabellon["duracion"] = pd.to_timedelta(df_pabellon["duracion"])

# Limpiar datos de operaciones quirúrgicas
df_pabellon_limpio = build_features.limpiar_datos_operaciones_quirurgicas(df_pabellon)
df_pabellon_limpio = df_pabellon_limpio.reset_index(names="id_int_q")

Una vez que se han limpiado todas las bases de datos, se procedera a unir GRD y Pabellon.

In [ ]:
# Fusionar GRD con datos de operaciones quirúrgicas
grd_sabanas_y_pabellon = build_features.fusionar_grd_con_operaciones_quirurgicas(
    df_grd_sabanas,
    df_pabellon_limpio,
    "RUT_LIMPIO",
    COLUMNAS_FECHA_SABANAS[0],
    COLUMNAS_FECHA_SABANAS[1],
)
grd_interno_y_pabellon = build_features.fusionar_grd_con_operaciones_quirurgicas(
    df_grd_interno,
    df_pabellon_limpio,
    "RUT_LIMPIO",
    COLUMNAS_FECHA_GRD_INT[0],
    COLUMNAS_FECHA_GRD_INT[1],
)

In [ ]:
grd_interno_y_pabellon.to_csv("../data/processed/grd_interno_y_pabellon.csv")

# 1. Control de Calidad de Union GRD y Pabellon

## 1.1 Cantidad de Pacientes

En teoria, los pacientes que aparezcan en Pabellon si o si deben estar en GRD.

In [ ]:
# Obtiene los pacientes unicos de cada base de dato
pacientes_pabellon = set(df_pabellon_limpio["ID_PACIENTE"].unique())
pacientes_GRD = set(df_grd_interno["RUT_LIMPIO"].unique())

In [ ]:
venn2([pacientes_GRD, pacientes_pabellon])
plt.show()

Por lo tanto, hay 10154 pacientes que estan en ambas bases de datos. GRD, como es de esperar, posee un mayor universo de pacientes. Sin embargo, existen 291 pacientes que solamente existen en la base de pabellon, y es necesario identificarlos:

In [ ]:
# Obtiene la cantidad de pacientes solo en pabellon
pacientes_solo_en_pabellon = pacientes_pabellon - pacientes_GRD
print(f"Hay {len(pacientes_solo_en_pabellon)} pacientes solamente en pabellon")

In [ ]:
# Obtiene todos los registros de operaciones de pacientes en pabellon
operaciones_de_pacientes_solo_en_pabellon = df_pabellon_limpio.query(
    "ID_PACIENTE.isin(@pacientes_solo_en_pabellon)"
)

print(
    f"De los {len(pacientes_solo_en_pabellon)} pacientes solamente en pabellon, estos se "
    f"realizaron {operaciones_de_pacientes_solo_en_pabellon.shape[0]} intervenciones quirurgicas"
)

In [ ]:
operaciones_validas = df_pabellon_limpio.shape[0]
print(f"Hay {operaciones_validas} operaciones validas en la base de datos de pabellon")

operaciones_unidas_con_grd = grd_interno_y_pabellon.shape[0]
print(f"Al unir GRD y Pabellon, se unieron {operaciones_unidas_con_grd} intervenciones quir.")

diferencia_pabellon_y_union_con_grd = operaciones_validas - operaciones_unidas_con_grd
print(f"Por lo tanto, al unir pabellon y GRD, hay {diferencia_pabellon_y_union_con_grd} int. q. "
      f"que quedaron sin unir")

print(f"De las {diferencia_pabellon_y_union_con_grd}, "
      f"{operaciones_de_pacientes_solo_en_pabellon.shape[0]} se realizaron los pacientes "
      f"que solamente se encontraron en pabellon")

print(f"Por lo tanto, hay 50 int. q. de pacientes que si estan en GRD, pero que se filtraron "
      f"por fecha")

Ahora, cuales fueron las intervenciones de pacientes que si estan en GRD, pero que no se unieron?

In [ ]:
# Obtiene operaciones fuera de egresos
ids_operaciones_unidas = grd_interno_y_pabellon["id_int_q"].unique()
operaciones_sin_unir_totales = df_pabellon_limpio.query("~id_int_q.isin(@ids_operaciones_unidas)")
operaciones_sin_unir_fuera_de_egreso = operaciones_sin_unir_totales.query(
    "~ID_PACIENTE.isin(@pacientes_solo_en_pabellon)"
)